<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/001password_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title パスワード生成アプリ

!pip install qrcode[pil] pyperclip

import secrets
import string
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import qrcode
from PIL import Image
import io

# パスワード生成（10個固定）
def generate_password(length, use_upper, use_lower, use_digits, use_symbols):
    char_pool = ""
    if use_upper:
        char_pool += string.ascii_uppercase
    if use_lower:
        char_pool += string.ascii_lowercase
    if use_digits:
        char_pool += string.digits
    if use_symbols:
        char_pool += string.punctuation
    if not char_pool:
        return ["⚠️ 少なくとも1種類の文字を選んでください。"]

    return [''.join(secrets.choice(char_pool) for _ in range(length)) for _ in range(10)]

# 強度評価
def assess_strength(pw, use_upper, use_lower, use_digits, use_symbols):
    score = 0
    score += len(pw) >= 12
    score += use_upper + use_lower + use_digits + use_symbols
    if score <= 2:
        return "🔴 弱い"
    elif score == 3:
        return "🟡 普通"
    else:
        return "🟢 強い"

# QRコード生成
def display_qr_code(data):
    qr = qrcode.make(data)
    buf = io.BytesIO()
    qr.save(buf, format='PNG')
    img = Image.open(buf)
    display(img)

# クリップボードにJSでコピー
def copy_to_clipboard_js(text):
    js_code = f"""
    async function copyText(text) {{
        try {{
            await navigator.clipboard.writeText(text);
            alert("✅ コピー完了: " + text);
        }} catch (err) {{
            alert("❌ コピー失敗: " + err);
        }}
    }}
    copyText("{text}");
    """
    display(Javascript(js_code))

# ボタン処理
def on_generate_clicked(b):
    with output:
        clear_output()
        passwords = generate_password(
            length_slider.value,
            upper_checkbox.value,
            lower_checkbox.value,
            digit_checkbox.value,
            symbol_checkbox.value
        )

        for i, pw in enumerate(passwords, 1):
            strength = assess_strength(
                pw,
                upper_checkbox.value,
                lower_checkbox.value,
                digit_checkbox.value,
                symbol_checkbox.value
            )

            copy_btn = widgets.Button(description="コピー", button_style='info')
            qr_btn = widgets.Button(description="QR表示", button_style='warning')

            def make_copy_handler(pw_text):
                return lambda x: copy_to_clipboard_js(pw_text)

            def make_qr_handler(pw_text):
                return lambda x: display_qr_code(pw_text)

            copy_btn.on_click(make_copy_handler(pw))
            qr_btn.on_click(make_qr_handler(pw))

            display(widgets.HBox([
                widgets.Label(f"パスワード {i}: {pw}   {strength}"),
                copy_btn,
                qr_btn
            ]))

# ウィジェット
length_slider = widgets.IntSlider(value=12, min=8, max=64, description='長さ:')
upper_checkbox = widgets.Checkbox(value=True, description='英大文字 (A-Z)')
lower_checkbox = widgets.Checkbox(value=True, description='英小文字 (a-z)')
digit_checkbox = widgets.Checkbox(value=True, description='数字 (0-9)')
symbol_checkbox = widgets.Checkbox(value=False, description='記号 (!@#...)')
generate_button = widgets.Button(description="パスワードを生成", button_style='success')
output = widgets.Output()

generate_button.on_click(on_generate_clicked)

# UI
display(widgets.VBox([
    widgets.HTML("<h3>🔐 パスワード自動生成アプリ</h3>"),
    length_slider,
    upper_checkbox,
    lower_checkbox,
    digit_checkbox,
    symbol_checkbox,
    generate_button,
    output
]))

# 📄 パスワード自動生成アプリ（Google Colab版）仕様書

## 1. アプリ概要
Google Colab 上で実行可能な、対話式のパスワード自動生成ツール。ユーザーの入力に応じて、強力なランダムパスワードを即時に生成・表示する。GUI的な操作も取り入れ、操作性を高める。

---

## 2. 使用環境

- **プラットフォーム**：Google Colab（Jupyter Notebook）
- **開発言語**：Python 3.x
- **主なライブラリ**：
  - `secrets`（セキュアな乱数生成）
  - `string`（文字種の取得）
  - `ipywidgets`（ユーザー入力用のウィジェット）
  - `pyperclip`（クリップボード機能）※任意

---

## 3. 機能一覧

### 3.1 パスワード生成
- 安全な乱数により、指定条件に基づいたパスワードを生成。
- 指定可能なオプション：
  - パスワード長（最小8文字、最大64文字）
  - 使用文字種（英大文字、英小文字、数字、記号）

### 3.2 ユーザー入力（ウィジェット使用）
- スライダーで文字数を選択
- チェックボックスで使用文字種を選択
- ボタンで生成をトリガー
- テキストボックスに結果を表示

### 3.3 パスワードのコピー（オプション）
- `pyperclip` を用いて生成したパスワードをクリップボードにコピーする機能（Colabの制限により注意が必要）

---

## 4. ユーザーインターフェース

### 4.1 入力
- `IntSlider`：パスワード長の選択（例：8～64）
- `Checkbox`：各文字種の使用可否を選択
- `Button`：生成開始

### 4.2 出力
- `Text`：生成結果の表示
- `Label`または`HTML`：パスワード強度の簡易表示（任意）

---

## 5. 処理フロー

1. ユーザーがウィジェットで設定を選択
2. 「生成」ボタン押下でパスワード生成関数が呼び出される
3. 選択された文字種と長さに基づいて、`secrets`でランダムな文字列を構成
4. 結果を画面に表示
5. （オプションで）コピー機能を呼び出し

---

## 6. 拡張機能（任意）

- パスワードの強度評価（例：長さや複雑さによる簡易レベル表示）
- 履歴保存（リストに保存して何個も生成できる）
- QRコード生成（パスワードをスマホ転送）

---

## 💡 追加してみたいアイデア（機能・UI・体験）

### 1. 🔍 **類似・危険なパスワード検出**
- `password123` のような有名で危険な文字列が含まれていたら警告！
- 例：「⚠️ このパスワードはよく使われていて危険です」

---

### 2. 📁 **CSV保存（ダウンロード）**
- 生成されたパスワードを `CSV` や `TXT` でダウンロードできるように。
- **メリット**：あとで使いたいときに便利。

```python
from google.colab import files
import pandas as pd

def download_passwords(passwords):
    df = pd.DataFrame({'Password': passwords})
    df.to_csv('generated_passwords.csv', index=False)
    files.download('generated_passwords.csv')
```

---

### 3. 🌙 **ライト / ダークモード切り替え（Colab風UI）**
- `ToggleButton` でテーマ切り替え。
- UIがちょっとおしゃれに！

---

### 4. ⏰ **期限つきパスワード**
- 一時的なパスワード（例：10分だけ有効）を生成。
- QRコードの下に「有効期限：10分」など表示。

---

### 5. 🤖 **AIによりおすすめパスワード提示（語呂合わせ風など）**
- 例：「Tokyo2025!」「Sushi_Rain7」みたいな、覚えやすくて強い系。
- 意味のある英単語＋記号＋数字などでカスタム生成。

---

### 6. 📱 **スマホ画面対応・レスポンシブUI**
- Google Colab でも、スマホで見やすいサイズに最適化（文字サイズなど）

---

### 7. 🔐 **保存機能（セッション内パスワード履歴）**
- 直近に生成したパスワードを履歴で下に表示
- 「あとでコピーしたい」ときに便利！

---

## 😎 こんな組み合わせも楽しいかも：

| 機能             | 目的                        |
|------------------|-----------------------------|
| QRコード＋期限つきPW | 一時的なアクセスコード用     |
| 危険パスワード警告   | セキュリティリスクを減らす     |
| 保存＋CSV DL     | 管理者や複数人用アカウント管理 |
| AI語呂PW生成     | 人間が覚えやすくする工夫       |

---